In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import pickle

In [21]:
import requests

headers = {
    'authority': 'zsurpubi.hu',
    'accept': '*/*',
    'sec-fetch-dest': 'empty',
    'x-requested-with': 'XMLHttpRequest',
    'request-id': '|NvdVJ.YQzbH',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'origin': 'https://zsurpubi.hu',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'referer': 'https://zsurpubi.hu/kereses/?keyword=&date_from=2019.+01.+01.&date_to=2019.+12.+31.&author=&category=&tags=',
    'accept-language': 'en-GB,en;q=0.9,hu-HU;q=0.8,hu;q=0.7,en-US;q=0.6'}

params = (
    ('keyword', ''),
    ('date_from', '2019. 01. 01.'),
    ('date_to', '2019. 12. 31.'),
    ('author', ''),
    ('category', ''),
    ('tags', ''),
)

data = {
  'search_more_articles': '1',
  'limit': '2929',
  'offset': '0'
}

response = requests.post('https://zsurpubi.hu/kereses/', headers=headers, params=params, data=data)

In [22]:
soup = BeautifulSoup(response.content, 'html.parser')

In [27]:
links = ["https://zsurpubi.hu/" + tag.find("a").get("href") for tag in soup.find_all("h2")]

In [42]:
links[1]

'https://zsurpubi.hu//cikk/9727-bodocs-tibor-aderes-ibizas-szilveszteres-parodiavideoja-eddig-mindent-visz/'

In [46]:
links[2]

'https://zsurpubi.hu//cikk/9723-brutalisan-no-a-szegenysegben-elo-nyugdijasok-szama-orbaneknak-azonban-csak-a-valasztasok-elotti-osztogatasok-fontosak/'

In [75]:
zsurpubi_dict = {}
for url in tqdm(links[:10]):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        datetime = soup.find("time").get("datetime")
        related_urls = []
        paragraphs = soup.find_all("p")
        for paragraph in paragraphs:
            if "MTI" in paragraph.text or " mti " in paragraph.text or "Mti" in paragraph.text:
                related_urls.append("MTI")
            if paragraph.find_all("a") is not None:
                for link in paragraph.find_all("a"):
                    if "http" in link.get('href'):
                        related_urls.append(link.get('href'))
                    else:
                        related_urls.append("https://zsurpubi.hu/" +  link.get('href'))
        zsurpubi_dict[url] = {
        "related_links" : related_urls,
        "publish_date" : datetime
    }